# Index

1. [Model](#1)
2. [Implementation](#2)
3. [WikiText 103 dataset processing](#3)
4. [WikiText 103 Benchmark](#4)

# 1. Model <a class="anchor" id="1"></a>

### Language Modeling

Un modelo de lenguaje busca calcular la probabilidad de observar una secuancia de tokens o palabras $w_{0}, \ldots, w_{N}$, usando la regla de la cadena de probabilidades la probabilidad conjunta de observar tal secuencia se descompone en la siguiente expresión
:
\begin{equation*}
p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N}p(w_{i}|\ldots, w_{i-1})
\end{equation*}

### N-grams 

Un modelo de N-grams posee una fuerte suposición sobre las probabilidades condicionales de una palabra y su historia ($p(w_{i}|\ldots, w_{i-1})$), así la probabilidad de obserbar la palabra $w_{i}$ depende de los N-1 palabras anteriores. Por ejemplo:

#### Unigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i})
\end{equation*}

#### Bigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i}| w_{i-1})
\end{equation*}

#### Trigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i}| w_{i-2}, w_{i-1})
\end{equation*}

### Trigrams with linear interpolation

El baseline a implementar consiste de un modelo de trigramas con interpolación lineal para evitar el problema de **zero-count**, este problema ocurre cuando la probabilidad de observar un trigrama es cero, lo que trae por consecuencia que la probabilidad conjunta sea cero, pues usando regla de la cadena y markov de segundo orden la probabilidad conjunta se descompone en el producto de los trigramas de la secuencia y basta que un trigrama tenga probabilidad cero para hechar a perder el cálculo de la probabilidad.

#### Trigrams
\begin{equation*}
p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N} p(w_{i}| w_{i-2}, w_{i-1})
\end{equation*}

#### Trigrams with linear interpolation

La probabilidad que entrega este modelo es la combinación convexa de las probabilidades que entregan los modelos unigrams, bigrams y trigrams.

\begin{align}
\nonumber
& p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N}\lambda_{1} p(w_{i}| w_{i-2}, w_{i-1})+\lambda_{2} p(w_{i}| w_{i-1})+\lambda_{3} p(w_{i})\\ \nonumber
& \lambda_{i}\geq 0, \; \lambda_{1}+\lambda_{2}+\lambda_{3}=1\ \nonumber
\end{align}

Los parámetros del modelo se estiman contando casos favorables versus totales, por ejemplo la probabilidad estimada de un trigrama viene dada por la siguiente expresión:
\begin{equation*}
q(w_{i}|w_{i-2}, w_{i}) = \frac{Count(w_{i-2}, w_{i-1}, w_{i})}{Count(w_{i-2}, w_{i-1})}
\end{equation*}


### Perplexity


La likelihood de un language model:

\begin{equation*}
L = \prod_{d=1}^{D}p(w_{1}, \ldots, w_{n_{d}}) = \prod_{d=1}^{D}\prod_{i=1}^{n_{d}}p(w_{i}|\ldots, w_{i-1})
\end{equation*}


Sea $D$ el número de documentos, $n_{d}$ el número de tokens del documento $d$, notar que los documentos son independientes.

Luego, la negative log-likelihood:

\begin{equation*}
NLL = \sum_{d=1}^{D}\sum_{i=1}^{n_{d}}-ln\big(p(w_{i}|\ldots, w_{i-1})\big) = \sum_{i=1}^{N}-ln\big(p(w_{i}|\ldots, w_{i-1})\big)
\end{equation*}

Donde, $N$ es el número de tokens presentes en el corpus.

\begin{equation*}
perplexity = e^{\frac{1}{N}\times NLL}
\end{equation*}

Por tanto, minimizar la perplexity es equivalente a minimizar NLL o maximizar LL.

En el caso del modelo de trigramas la perplexity queda:

\begin{equation*}
perplexity = exp\left(\frac{1}{N}\sum_{i=1}^{N}-ln\big(\lambda_{1} q(w_{i}| w_{i-2}, w_{i-1})+\lambda_{2} q(w_{i}| w_{i-1})+\lambda_{3} q(w_{i})\big)\right)
\end{equation*}

# 2. Implementation <a class="anchor" id="2"></a>

In [1]:
%%capture
import os
import re
import shutil
import numpy as np
import pandas as pd
from time import time
from collections import defaultdict
from scipy.stats import dirichlet

In [2]:
class Trigrams():     

    def fit(self, corpus):
        """
        Ajusta el modelo en el corpus de entrenamiento, guarda los parámetros en atributos.

        Parameters:
            corpus: {list of str}, shape (corpus_size) 
                corpus de entrenamiento tokenizado.
        Returns:
            self: object
        """
        #Número de documentos
        N = len(corpus) 
       
        ##Unigrams
        model1 = defaultdict(lambda: 0)
        #contar unigramas
        for doc in corpus:
            doc = tokenizer(doc)
            for word in doc:
                model1[word]+=1
        n_tokens = np.array(list(model1.values())).sum()
        vocabulary = list(model1.keys())
        #probabilidades unigramas: dividir la frecuencia por el total de tokens
        for word in vocabulary:
            model1[word] = model1[word]/n_tokens
        model1['</s>']= N/n_tokens
        
        
        ##Bigrams
        model2 = defaultdict(lambda: defaultdict(lambda: 0))
        #probabiliades bigramas
        for doc in corpus:
            doc = tokenizer(doc)
            doc = ['<s>']+doc+['</s>']#start and end symbol
            n = len(doc)#largo del documento
            for i in range(1, n):   
                word_i_1 = doc[i-1]
                word_i = doc[i]
                if i == 1:#primera palabra
                    q_w1 = N
                else:#segunda palabra en adelante
                    q_w1 = model1[word_i_1]*n_tokens
                
                model2[word_i_1][word_i]+=1/q_w1

        ##Trigrams
        model3 = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
        #probabiliades trigramas
        for doc in corpus:
            doc = tokenizer(doc)
            doc = ['<s>', '<s>']+doc+['</s>']#start and end symbol
            n = len(doc) #largo del documento
            for i in range(2, n):
                word_i_2 = doc[i-2]
                word_i_1 = doc[i-1]
                word_i = doc[i]
                if i == 2:#primera palabra
                    q_w1_w2 = N
                elif i == 3:#segunda palabra
                    q_w2 = N
                    q_w1_w_2 = model2[word_i_2][word_i_1]*q_w2
                else:#tercera palabra en adelante
                    q_w2 = model1[word_i_2]*n_tokens
                    q_w1_w2 = model2[word_i_2][word_i_1]*q_w2
                    
                model3[word_i_2][word_i_1][word_i]+=1/q_w1_w2
        
        #guardar parámetros en atributos y el vocabulario
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.vocabulary = vocabulary

    def predict_proba_trigam(self, lamb, trigram):
        """
        Calcula la probabilidad de un trigrama usando interpolación lineal.
            - p(w3|w1,w2) = lamb_1*q(w3|w1,w2)+lamb_2*q(w3|w2)+lamb_3*q(w3)
            - lamb>=0, lamb_1+lamb_2+lamb_3=1
        Parameters:
            lamb: array, shape = 3
                hiperparámetros del modelo, cada componente debe ser positivo y deben sumar 1.
            trigram: tuple of str, shape = 3
                trigrama, de la forma (w1, w2, w3), donde w3 es el último token de la secuencia.

        Returns:
            score: float
                probabilidad del trigrama
        """
        w1, w2, w3 = trigram
        proba = lamb[0]*self.model3[w1][w2][w3]+lamb[1]*self.model2[w2][w3]+lamb[2]*self.model1[w3]
        return proba
        
    def get_perplexity(self, lamb, corpus):

        """
        Obtiene la perplexity del modelo en un conjunto de validación
            -perplexity = exp(NLL/N), donde NLL es la negative-log-likelihood del modelo y 
             N el número de trigramas en el corpus.

        Parameters:
            - corpus: {list of list of str}, shape (corpus_size) 
                  corpus de validación tokenizado.
            - lamb: {array}, shape =3
                  hiperparámetros del modelo, cada componente debe ser positivo y deben sumar 1.
        Returns:
            score: float
                perplexity
        """

        N = 0 #contador de trigramas del conjunto de validación
        NLL = 0 #negative-log-likelihood
        for doc in corpus:
            doc = ['<s>', '<s>']+doc+['</s>']#start and end symbol
            n = len(doc) #largo del documento
            for i in range(2, n):
                word_i_2 = doc[i-2]
                word_i_1 = doc[i-1]
                word_i = doc[i]
                trigram = word_i_2, word_i_1, word_i, 
                trigram_proba = self.predict_proba_trigam(lamb, trigram)
                NLL = NLL - np.log(trigram_proba) 
                N = N+1
        perplexity = np.exp(NLL/N)

        return perplexity

# 3. WikiText 103 dataset processing <a class="anchor" id="3"></a>

Leemos los archivos a partir de un .zip, el .zip cuenta con tres archivos planos, train, valid y test. De cada archivo se extraen los documentos y son guardados en una lista dentro de un diccionario.

In [3]:
def get_vocabulary(dataset):
    vocabulary = set()
    for doc in dataset:
        set_doc = set(doc.split())
        vocabulary.update(set_doc)
    return vocabulary

def tokenizer(doc, vocabulary=None):
    tokens = doc.split()
    if vocabulary:
        tokens = [word for word in tokens if word in vocabulary]
    return tokens

def remove_out_of_vocabulary(vocabulary, dataset):
    new_dataset = []
    deleted = 0
    
    for doc in dataset:
        tokens =  tokenizer(doc, vocabulary)
        new_dataset.append(tokens)
    
    return new_dataset

In [4]:
#importar datasets
shutil.unpack_archive('../wikitext-103-v1.zip', extract_dir='dataset')
working_dir = os.path.join(os.getcwd(), 'dataset', 'wikitext-103')
wikitext_files = os.listdir(working_dir)

wiki = {}
for wikitext_file in wikitext_files:
    with open(os.path.join(working_dir, wikitext_file), encoding='utf-8') as data_file:
        name = wikitext_file.split('.')[1]
        corpus = []
        for index, line in enumerate(data_file):
            # filtrar lineas vacías y headers
            if len(line) < 3 or line[1] == '=':
                continue
            else:
                corpus.append(line.strip())
        #list of str: dataset no tokenizado        
        wiki[name] = corpus
shutil.rmtree('dataset')

In [12]:
wiki['test'][3]

'He had a recurring role in 2003 on two episodes of The Bill , as character " Connor Price " . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . Boulter starred as " Darren " , in the 2005 theatre productions of the Philip Ridley play Mercury Fur . It was performed at the Drum Theatre in Plymouth , and the <unk> Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . Boulter received a favorable review in The Daily Telegraph : " The acting is shatteringly intense , with wired performances from Ben Whishaw ( now unrecognisable from his performance as Trevor Nunn \'s Hamlet ) , Robert Boulter , Shane Zaza and Fraser Ayres . " The Guardian noted , " Ben Whishaw and Robert Boulter offer tenderness amid the savagery . "'

In [5]:
len(wiki['train'])

859532

In [6]:
#Extraemos el vocabulario de train y removemos los tokens de validation y test que no estan en él
vocabulary = get_vocabulary(wiki['train'])
wiki['valid'] = remove_out_of_vocabulary(vocabulary, wiki['valid']) 
wiki['test'] = remove_out_of_vocabulary(vocabulary, wiki['test']) 

## 4. WikiText 103 Benchmark <a class="anchor" id="4"></a>

Entrenamos el modelo de trigramas en el conjunto de entrenamiento.

In [7]:
wikitext_model = Trigrams()
%time wikitext_model.fit(wiki['train'])

CPU times: user 8min 28s, sys: 5.02 s, total: 8min 34s
Wall time: 8min 33s


Evaluamos el desempeño del modelo de trigramas en validation para 100 configuraciones diferentes de $\lambda$.

In [8]:
ti = time()
#una muestra que distribuye dirichlet es un vector donde cada componente es no negativo y suman 1 
pdf = dirichlet(alpha = [64, 16, 4]) #muestras que tienden a satisfacer lambda1>lambda2>lambda3
lambdas = pdf.rvs(size=100, random_state=0) #generar 100 muestras
#computar perplexity para las 100 configuraciones en validación
perplexity = []
for i in range(len(lambdas)):
    perplexity.append(wikitext_model.get_perplexity(lambdas[i], wiki['valid']))
tf = time()
print(f"Wall time: {tf-ti}")

Wall time: 108.43068838119507


In [12]:
validation_scores = pd.DataFrame({'lambda':pd.Series(lambdas.tolist()), 'perplexity':pd.Series(perplexity)})
validation_scores.sort_values('perplexity').head() #mostramos las primeras 5 

,lambda,perplexity
4,"[0.6155053151143111, 0.2494869060497376, 0.135...",200.398335
5,"[0.6389284514853635, 0.27312456407959157, 0.08...",200.978871
95,"[0.662545908458533, 0.24844956753710032, 0.089...",203.184069
12,"[0.693068868830631, 0.23634674049083365, 0.070...",206.868809
92,"[0.6881357187938214, 0.20199917444681775, 0.10...",206.879514


Tomamos la configuración con mejor perplexity (el valor mínimo) y evaluamos en test obteniendose una perplexity de 205.44.

In [13]:
best_configurations = validation_scores.sort_values('perplexity', ascending=True).head()
best_lambda = best_configurations['lambda'].iloc[0]
best_configurations

,lambda,perplexity
4,"[0.6155053151143111, 0.2494869060497376, 0.135...",200.398335
5,"[0.6389284514853635, 0.27312456407959157, 0.08...",200.978871
95,"[0.662545908458533, 0.24844956753710032, 0.089...",203.184069
12,"[0.693068868830631, 0.23634674049083365, 0.070...",206.868809
92,"[0.6881357187938214, 0.20199917444681775, 0.10...",206.879514


In [14]:
perplexity_test = wikitext_model.get_perplexity(best_lambda, wiki['test'])
perplexity_test

205.44248549841095